In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
!unzip 'Dataset.zip'

Archive:  Dataset.zip
   creating: Dataset/body/
   creating: Dataset/body/training/
   creating: Dataset/body/training/00-front/
  inflating: Dataset/body/training/00-front/0001.jpeg  
  inflating: Dataset/body/training/00-front/0002.JPEG  
  inflating: Dataset/body/training/00-front/0003.JPEG  
  inflating: Dataset/body/training/00-front/0004.JPEG  
  inflating: Dataset/body/training/00-front/0005.JPEG  
  inflating: Dataset/body/training/00-front/0006.JPEG  
  inflating: Dataset/body/training/00-front/0007.JPEG  
  inflating: Dataset/body/training/00-front/0008.jpeg  
  inflating: Dataset/body/training/00-front/0009.JPEG  
  inflating: Dataset/body/training/00-front/0010.JPEG  
  inflating: Dataset/body/training/00-front/0011.JPEG  
  inflating: Dataset/body/training/00-front/0012.jpeg  
  inflating: Dataset/body/training/00-front/0013.JPEG  
  inflating: Dataset/body/training/00-front/0014.JPEG  
  inflating: Dataset/body/training/00-front/0015.JPEG  
  inflating: Dataset/body/trai

In [3]:
imageSize = [224, 224]

trainPath = r"/content/Dataset/body/training"

testPath = r"/content/Dataset/body/validation"


In [4]:
# adding preprocessing layers to the front of vgg

vgg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [5]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [6]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)

In [7]:
prediction = Dense(3, activation='softmax')(x)

In [8]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [13]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=979//10,
  validation_steps=171//10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/25
97/97 [==============================] - 31s 210ms/step - loss: 1.3787 - accuracy: 0.5046 - val_loss: 0.9747 - val_accuracy: 0.6412
Epoch 2/25
97/97 [==============================] - 19s 192ms/step - loss: 0.8290 - accuracy: 0.6811 - val_loss: 0.8946 - val_accuracy: 0.6706
Epoch 3/25
97/97 [==============================] - 19s 192ms/step - loss: 0.5530 - accuracy: 0.7936 - val_loss: 1.3514 - val_accuracy: 0.6059
Epoch 4/25
97/97 [==============================] - 19s 192ms/step - loss: 0.5447 - accuracy: 0.7905 - val_loss: 0.8740 - val_accuracy: 0.6588
Epoch 5/25
97/97 [==============================] - 19s 193ms/step - loss: 0.4041 - accuracy: 0.8431 - val_loss: 0.9125 - val_accuracy: 0.6706
Epoch 6/25
97/97 [==============================] - 19s 193ms/step - loss: 0.3447 - accuracy: 0.8669 - val_loss: 0.9191 - val_accuracy: 0.6706
Epoch 7/25
97/97 [==============================] - 19s 193ms/step - loss: 0.2745 - accuracy: 0.9061 - val_loss: 0.9782 - val_accuracy: 0.6882

In [32]:
#save the model
model.save('body.h5')

# Testing

In [16]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np

In [26]:
#load one random image from local system
img=image.load_img(r'/content/Dataset/body/training/02-side/0001.JPEG',target_size=(224,224))

In [27]:
#convert image to array format
x=image.img_to_array(img)

In [28]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [29]:
img_data.shape

(1, 224, 224, 3)

In [30]:
model.predict(img_data)

array([[3.4051514e-04, 1.9898226e-04, 9.9946052e-01]], dtype=float32)

In [31]:
output=np.argmax(model.predict(img_data), axis=1)
output

array([2])

# model 2

In [33]:
imageSize = [224, 224]

trainPath = r"/content/Dataset/level/training"

testPath = r"/content/Dataset/level/validation"

In [34]:
# adding preprocessing layers to the front of vgg

vgg1 = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

In [35]:
# don't train existing weights
for layer in vgg1.layers:
  layer.trainable = False

In [36]:
# our layers - you can add more if you want
x = Flatten()(vgg1.output)

In [37]:
prediction = Dense(3, activation='softmax')(x)

In [38]:
# create a model object
model1 = Model(inputs=vgg1.input, outputs=prediction)

In [39]:
# tell the model what cost and optimization method to use
model1.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [40]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [41]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size = 10,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 10,
                                            class_mode = 'categorical')

Found 979 images belonging to 3 classes.
Found 171 images belonging to 3 classes.


In [42]:
r = model1.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=979//10,
  validation_steps=171//10)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


97/97 [==============================] - 20s 197ms/step - loss: 1.0795 - accuracy: 0.5738 - val_loss: 1.3474 - val_accuracy: 0.6059
Epoch 2/25
97/97 [==============================] - 19s 194ms/step - loss: 0.7994 - accuracy: 0.7245 - val_loss: 1.4542 - val_accuracy: 0.6176
Epoch 3/25
97/97 [==============================] - 19s 192ms/step - loss: 0.6969 - accuracy: 0.7379 - val_loss: 0.9930 - val_accuracy: 0.6353
Epoch 4/25
97/97 [==============================] - 19s 193ms/step - loss: 0.5025 - accuracy: 0.8029 - val_loss: 1.0322 - val_accuracy: 0.6588
Epoch 5/25
97/97 [==============================] - 19s 194ms/step - loss: 0.3996 - accuracy: 0.8338 - val_loss: 1.3540 - val_accuracy: 0.5647
Epoch 6/25
97/97 [==============================] - 19s 193ms/step - loss: 0.3990 - accuracy: 0.8421 - val_loss: 1.1297 - val_accuracy: 0.6353
Epoch 7/25
97/97 [==============================] - 19s 193ms/step - loss: 0.4098 - accuracy: 0.8483 - val_loss: 1.2793 - val_accuracy: 0.6235
Epoch 8/25

In [43]:
#save the model
model.save('level.h5')